# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

print(geoapify_key)

590996ee256942c08eb0b4bf3250e556


In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vorgashor,67.5833,63.9500,244.71,97,9,2.53,RU,04-02-2025
1,1,grytviken,-54.2811,-36.5092,278.83,93,100,2.92,GS,03-02-2025
2,2,jamestown,42.0970,-79.2353,281.95,76,75,5.14,US,03-02-2025
3,3,pico truncado,-46.7949,-67.9573,292.01,15,39,6.89,AR,03-02-2025
4,4,blackmans bay,-43.0167,147.3167,295.22,69,77,2.61,AU,04-02-2025


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [40]:
# Narrow down cities that fit criteria and drop any results with null values
print(city_data_df.shape)

# sorted_df = city_data_df.sort_values(by="Cloudiness", ascending=True)
# sorted_df.head()

narrowed_city_df = city_data_df.loc[
    # (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df.head()  #.sort_values("Cloud", ascending=True)

(561, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,kodiak,57.7900,-152.4072,274.88,75,0,2.57,US,03-02-2025
17,17,bredasdorp,-34.5322,20.0403,291.61,73,0,1.42,ZA,03-02-2025
19,19,lata,40.1629,-8.3327,278.36,85,0,1.29,PT,03-02-2025
29,29,sandnessjoen,66.0217,12.6316,272.21,55,0,3.60,NO,03-02-2025
30,30,plettenberg bay,-34.0527,23.3716,292.31,87,0,0.71,ZA,03-02-2025


### Step 3: Create a new DataFrame called `hotel_df`.

In [41]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
11,kodiak,US,57.7900,-152.4072,75,
17,bredasdorp,ZA,-34.5322,20.0403,73,
19,lata,PT,40.1629,-8.3327,85,
29,sandnessjoen,NO,66.0217,12.6316,55,
30,plettenberg bay,ZA,-34.0527,23.3716,87,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [42]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kodiak - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
lata - nearest hotel: Residencial Botânico
sandnessjoen - nearest hotel: Scandic Syv Søstre
plettenberg bay - nearest hotel: Milkwood Manor
sitka - nearest hotel: Aspen Suites Hotel
hermanus - nearest hotel: Aloe guest house
mao - nearest hotel: Hevresac
port elizabeth - nearest hotel: Waterford Hotel
crane - nearest hotel: No hotel found
ghadamis - nearest hotel: فندق عين الفارس
cabo san lucas - nearest hotel: Comfort Rooms
monsummano terme - nearest hotel: No hotel found
beipiao - nearest hotel: No hotel found
saipan - nearest hotel: Chalan Kanoa Beach Hotel
chela - nearest hotel: 宕昌龙海大酒店
bam - nearest hotel: هتل جهانگردی بم
hede - nearest hotel: No hotel found
tura - nearest hotel: Hotel Polo Orchid
lucas - nearest hotel: Hawthorn Suites by Wyndham Holland/Toledo Area
general conesa - nearest hotel: Gran Hotel Conesa
turbat - nearest hotel: No hotel found
peniche - nearest hotel: 

,City,Country,Lat,Lng,Humidity,Hotel Name
11,kodiak,US,57.7900,-152.4072,75,No hotel found
17,bredasdorp,ZA,-34.5322,20.0403,73,Victoria Hotel
19,lata,PT,40.1629,-8.3327,85,Residencial Botânico
29,sandnessjoen,NO,66.0217,12.6316,55,Scandic Syv Søstre
30,plettenberg bay,ZA,-34.0527,23.3716,87,Milkwood Manor
36,sitka,US,57.0531,-135.3300,30,Aspen Suites Hotel
51,hermanus,ZA,-34.4187,19.2345,75,Aloe guest house
63,mao,ES,39.8885,4.2658,93,Hevresac
66,port elizabeth,ZA,-33.9180,25.5701,93,Waterford Hotel
72,crane,US,31.3974,-102.3501,16,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [43]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)